<a href="https://colab.research.google.com/github/Bahramif/Deep-Trading/blob/master/FLIPP_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-cloud-vision

Requirement already up-to-date: google-cloud-vision in /usr/local/lib/python3.6/dist-packages (0.39.0)


In [2]:
!pip install requests


In [3]:
from google.colab import files
uploaded = files.upload()

Saving My First Project-b4b48fb4c80a.json to My First Project-b4b48fb4c80a.json


In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="My First Project-b4b48fb4c80a.json"

In [0]:
Meat=['meat','fish','salmon','trout','tilapia','ground beef','steak','sirloin','ham','trout','tenderloin']
Grains_bread=['pasta','rice','bread']
oil=['butter','oil']
Diary=['egg','cheese','yogurt','milk']
Produce=['onion','garlic']


fruits=['apple', 'apricot','banana','blackberry','cherry','grapefruit','mango','kiwi','orange','papaya','passion fruit'
,'peach','pear','pea','pineapple','plum','watermelon','raspberry','blueberry','grape']

Vegetables=['asparagus','cabbage','avocado','bean','broccoli','corn','cucumber','carrot','eggplant','lemon','lettuce','mushroom','potatoe',
            'tomato','green pepper','asparagus','brussels sprout','parsley','oregano','thyme','rosemary','basil','lavender']
All=fruits+Vegetables+Produce+Diary+oil+Grains_bread+Meat

## Scrape Flipp

In [0]:
import requests
import pandas as pd

BASE_URL = 'https://flipp.com'
BACKEND_URL = 'https://backflipp.wishabi.com/flipp'
SEARCH_URL = '%s/items/search' % BACKEND_URL
ITEM_URL = '%s/items/' % BACKEND_URL

# def scrape_item(item_id):
#     return requests.get(
#         "%s/%s" % (ITEM_URL, item_id,)
#     ).json()

def search(query, postal_code, locale):
    data = requests.get(
        SEARCH_URL,
        params = {
            'q': query,
            'postal_code': postal_code,
            'locale':locale
        }
    ).json()

    return [
#             scrape_item(x.get('flyer_item_id'))

            x for x in data.get('items')
    ]




In [0]:
# testing search function
search_result = search('Milk','l6a3r7', 'en-ca')
columns = list(search_result[0].keys())


Convert the product list to dataframe

In [8]:

query_results=list()
df=pd.DataFrame()
for items in All:
  try:
    final_dict = {key:[] for key in columns}
    keys = final_dict.keys()
    query_results=search(items,'l6a3r7', 'en-ca')
    for q in query_results:
       for key in keys:
          final_dict[key].append(q[key])
    final_dict2=pd.DataFrame(final_dict)
    df=df.append(final_dict2)
    row=final_dict2.shape[0]
    df.index=range(df.shape[0])
    df.loc[df.shape[0]-row:,'product']=items
  except:
    continue

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
df_sort=df.sort_values(['merchant_name'])
mechant_name=df_sort.merchant_name.unique()
df_sort["current_price"] = df_sort["current_price"].apply(pd.to_numeric)
df_sort2=df_sort[df_sort.current_price>0]

Select the grocery sores from the merchant list

In [0]:
food_store=[]
for x in mechant_name:
  if 'food' in x.lower():
    food_store.append(x)
  if 'supermarket' in x.lower():
    food_store.append(x)
  if 'superstore' in x.lower():
    food_store.append(x)
  if 'mart' in x.lower():
    food_store.append(x)
  if 'fresh' in x.lower():
    food_store.append(x)
  if x in ['Walmart','Zehrs','Starsky','Sobeys','Price Chopper','No Frills','Longos','Metro','Loblaws','Healthy Planet','Giant Tiger']:
    food_store.append(x)
    

In [0]:
df_sort_food=df_sort2[df_sort2.merchant_name.isin(food_store)]
df_sort_food.index=range(df_sort_food.shape[0])
df_sort_food2=df_sort_food

In [12]:
df_sort_food2.shape

(1117, 23)

##Image processing and create labels for images

In [13]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate

import io
import requests
from PIL import Image
import matplotlib.pyplot as plt  
def Labeled_image(url):
  data = requests.get(url).content
  image = types.Image(content=data)
  img = Image.open(io.BytesIO(data))
  response = client.label_detection(image=image)
  labels = response.label_annotations
  Label_img=list()
  #   plt.imshow(img)
  #   plt.show()
  for label in labels:
    Label_img.append(label.description)
  return Label_img

df_sort_food2['labels']=df_sort_food2['clean_image_url'].apply(lambda x:Labeled_image(str(x)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Filter for relevant photos

In [14]:
for i in df_sort_food2.index:
  df_sort_food2.loc[i,'Relevant']=pd.Series(df_sort_food2.loc[i,'labels']).isin(['Fruit','Fish','Meat','Vegetable','Ingredient','Dairy','Oil']).any()


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [0]:
df_sort_food2=df_sort_food2[df_sort_food2.Relevant==True]
# df_sort_food2.to_pickle('./flipp.pkl')

In [20]:
from google.colab import files
uploaded = files.upload()

Saving flipp_product.csv to flipp_product (1).csv


In [0]:
import pandas as pd
df_sort_food2=pd.read_csv('flipp_product.csv')

##OCR flyers for weight analysis

In [19]:
def detect_text_url(uri):
  try:
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    response = client.text_detection(image=image)
    texts = response.text_annotations
#     print('Texts:')
    return texts[0].description
  except:
    return 'error'
  

for i in df_sort_food2.index:
  if df_sort_food2.loc[i,"Relevant"]==True:
    df_sort_food2.loc[i,"ocr_name"]=detect_text_url(df_sort_food2.loc[i,"clean_image_url"])


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [0]:
import re
df_sort_food2['vagetables']=[any([i == 'Vegetable' for i in x]) for x in df_sort_food2.labels ]
df_sort_food2['Fruit']=[any([i == 'Fruit' for i in x]) for x in df_sort_food2.labels ]
df_sort_food2['meat_chicken']=[any(re.findall('chicken',x.lower()))  for x in df_sort_food2.ocr_name ]
df_sort_food2['meat_steak']=[any(re.findall('steak',x.lower()))  for x in df_sort_food2.ocr_name ]
df_sort_food2['meat_ground']=[any(re.findall('ground',x.lower()))  for x in df_sort_food2.ocr_name ]
df_sort_food2['meat_tenderloin']=[any(re.findall('tenderloin',x.lower()))  for x in df_sort_food2.ocr_name ]
df_sort_food2['meat_tenderloin']=[any(re.findall('sirloin',x.lower()))  for x in df_sort_food2.ocr_name ]

Extract weight numbers from ocr text

In [0]:
import re
import inflect
p = inflect.engine()
def Weight(name,product):
  pattern1 = re.compile(r'((?:[0-9]*\s*Pack)'
    r'|(?:[0-9.]+/*\s*/*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b))' , re.IGNORECASE)
  pattern2 = re.compile(r'('+str(p.plural(product))
                       +r'|'  +str(p.singular_noun(product))
                       +r'|' + str(product) +r')', re.IGNORECASE)
 
  pattern3 = re.compile(r'('+str(p.plural(product))
   +r'|' + str(product)+r'|((?:[0-9]*\s*Pack)'
    r'|(?:[0-9.]+/*\s*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b)))'
                       , re.IGNORECASE)
  return (re.findall(pattern1, name),re.findall(pattern2, name),re.findall(pattern3, name))

In [0]:
import numpy as np
def right_item(weight,product):
  r=weight
  D=np.array(r[2][:])
  try:
#   print(D[:,0])
    p.plural(product) in list(D[:,0])  

    o=0
    list2=list(map(lambda x:x.lower(),list(D[:,0])))

    list1=list(map(lambda x:x.lower(),list(r[0])))
    if p.plural(product.lower()) in list2:
      prod=p.plural(product).lower()
    if product.lower() in list2:
      prod=product.lower()

    try:
      x=list2.index(prod)
    except:
      x=0
      o=1
    if len(r[0])>1:
      while o==0 and x<=len(list2)-2:
        x+=1
        if list2[x] in list1:
          o=1
        
      if len(r[0])==1 or(o==0):
        x=0
      out=list2[x]

    if len(r[0])==1:
      out=list1[0]
    if len(r[0])==0:
      out='NaN'
  except:
    out='NaN'
  return out
  



In [28]:
i=6
r=Weight(df_sort_food2.ocr_name[i],df_sort_food2.loc[i,"product"])
product=df_sort_food2.loc[i,'product'].lower()
# right_item(list_weight,product)
# product

D=np.array(r[2][:])


o=0
list2=list(map(lambda x:x.lower(),list(D[:,0])))

list1=list(map(lambda x:x.lower(),list(r[0])))
if p.plural(product.lower()) in list2:
  prod=p.plural(product).lower()
if product.lower() in list2:
  prod=product.lower()

try:
  x=list2.index(prod)
except:
  x=0
  o=1
if len(r[0])>1:
  while o==0 and x<=len(list2)-2:
    x+=1
    if list2[x] in list1:
      o=1

  if len(r[0])==1 or(o==0):
    x=0
  out=list2[x]

if len(r[0])==1:
  out=list1[0]
if len(r[0])==0:
  out='NaN'

IndexError: ignored

In [35]:
Weight(df_sort_food2.ocr_name[9],df_sort_food2.loc[9,"product"])

([], [], [])

In [36]:
df_sort_food2.loc[9,"clean_image_url"]

'http://f.wishabi.net/page_items/164012182/1566221639/extra_large.jpg'

In [154]:
df_sort_food2.ocr_name[13]

'3 LB\nBAG\n349\n3 lb Bagged Seedless Oranges\nProduct of South Africa\nHoneydew Melons Product of USA\nOranges ou melon-miel\n'

In [0]:
# import re
# import inflect
# p = inflect.engine()
# def Weight(name,product):
#   pattern1 = re.compile(r'((?:[0-9]*\s*Pack)'
#     r'|(?:[0-9.]+/*\s*/*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b))' , re.IGNORECASE)
#   pattern2 = re.compile(r'(/*str(p.plural(product))'
#                        +r'/*?:|'  +str(p.singular_noun(product))
#                        +r'/*?:|' + str(product) +r')', re.IGNORECASE)
 
#   pattern3 = re.compile(r'('+str(p.singular_noun(product))
#    +r'|' + str(product)+r'|((?:[0-9]*\s*Pack)'
#     r'|(?:[0-9.]+/*\s*/*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b)))'
#                        , re.IGNORECASE)
#   return (re.findall(pattern1, name),re.findall(pattern2, name),re.findall(pattern3, name))

##Extract weights from ocr text

In [23]:
i=13
list_weight=Weight(df_sort_food2.ocr_name[i],df_sort_food2.loc[i,"product"])
product=df_sort_food2.loc[i,'product'].lower()
right_item(list_weight,product)
product

'steak'

In [24]:
for i in df_sort_food2.index:
  list_weight=Weight(df_sort_food2.ocr_name[i],df_sort_food2.loc[i,"product"])
  product=df_sort_food2.loc[i,'product'].lower()
  
  if len(list_weight[0])==0 and len(list_weight[1])==0 :
    df_sort_food2.loc[i,'weight']='NaN'
  elif len(list_weight[0])==0 and len(list_weight[1])>=1 :
    df_sort_food2.loc[i,'weight']=1
  elif len(list_weight[0])>=1 and len(list_weight[1])==0 :
    df_sort_food2.loc[i,'weight']=right_item(list_weight,product)
  elif len(list_weight[0])>=1 and len(list_weight[1])>=1 :
    df_sort_food2.loc[i,'weight']=right_item(list_weight,product)  



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [0]:
for i in df_sort_food2.index:
  
  df_sort_food2.loc[i,'weight_num']=0  
  df_sort_food2.loc[i,'price_100']=0
  df_sort_food2.loc[i,'weight2']=0  
  try:
    if df_sort_food2.loc[i,'weight']== 1:
      df_sort_food2.loc[i,'weight_num']=1  
    elif not any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'kg' :
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0])*1000
    elif any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'kg' :
      df_sort_food2.loc[i,'price_100']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-3])[0] )/10
    elif not any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-1].lower()=='g' and df_sort_food2.loc[i,'weight'][-2:].lower() != 'kg':  
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-1])[0] )
    elif df_sort_food2.loc[i,'weight'][-1].lower()=='l' and df_sort_food2.loc[i,'weight'][-2:].lower()=='ml':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )
    elif df_sort_food2.loc[i,'weight'][-1].lower()=='l' and df_sort_food2.loc[i,'weight'][-2:].lower()!='ml':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-1])[0] )*1000
    elif not any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'lb':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )*453.592
    elif any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'lb':
      df_sort_food2.loc[i,'price_100']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )/4.53592
    elif df_sort_food2.loc[i,'weight'][-2:].lower() == 'oz':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )*28.3495
    elif df_sort_food2.loc[i,'weight'][-4:].lower() == 'pack':
      df_sort_food2.loc[i,'weight_num']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'])[0] )  
  except:
    df_sort_food2.loc[i,'weight2']==0
  if df_sort_food2.loc[i,'price_100']==pd.np.nan and df_sort_food2.loc[i,'weight2']>0:
    df_sort_food2.loc[i,'price_100']=df_sort_food2.loc[i,'current_price']/(df_sort_food2.loc[i,'weight2']/100)
  

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [57]:
df_sort_food2['price_100'] 

0             NaN
2        0.439000
5             NaN
6             NaN
8        1.210000
9             NaN
10            NaN
11            NaN
12            NaN
13            NaN
14       0.328000
15            NaN
16            NaN
17            NaN
18            NaN
20       0.373000
21            NaN
22       0.328000
25            NaN
26            NaN
27            NaN
29            NaN
31            NaN
33            NaN
34       0.220462
35       1.321000
36            NaN
38            NaN
39            NaN
42       0.373000
          ...    
1069          NaN
1070          NaN
1071          NaN
1072     1.761000
1075          NaN
1077          NaN
1078          NaN
1079          NaN
1081          NaN
1082          NaN
1086     3.305000
1089          NaN
1090          NaN
1091          NaN
1092          NaN
1093          NaN
1094          NaN
1097          NaN
1098    28.400000
1099          NaN
1101          NaN
1102          NaN
1103          NaN
1104          NaN
1105      

In [55]:
df_sort_food2[df_sort_food2.price_100==0].count()

_L1                   0
bottom                0
clean_image_url       0
clipping_image_url    0
current_price         0
flyer_id              0
flyer_item_id         0
item_weight           0
left                  0
merchant_id           0
merchant_logo         0
merchant_name         0
name                  0
post_price_text       0
pre_price_text        0
premium               0
product               0
right                 0
sale_story            0
score                 0
top                   0
valid_from            0
valid_to              0
labels                0
Relevant              0
ocr_name              0
weight                0
weight_num            0
price_100             0
weight2               0
dtype: int64

In [0]:
df_sort_food3=df_sort_food2[df_sort_food2.Relevant==True]

In [0]:
df_agg = df_sort_food2.groupby(['merchant_name','product'],observed=True)['merchant_name','product','current_price','weight2','weight_num','price_100','clean_image_url'].agg({'price_100':min})

In [0]:
pd.set_option('max_colwidth', 200)

In [228]:
df_sort_food2[['merchant_name','product','current_price','weight2','weight_num','price_100','clean_image_url']].sort_values(['merchant_name','product','price_100'])

,merchant_name,product,current_price,weight2,weight_num,price_100,clean_image_url
31,Chalo FreshCo,apple,1.29,NaN,0.0,0.284000,http://f.wishabi.net/page_items/163299386/1564749343/extra_large.jpg
3,Chalo FreshCo,apricot,4.99,3000.000,0.0,0.166333,http://f.wishabi.net/page_items/164122515/1565366418/extra_large.jpg
6,Chalo FreshCo,apricot,3.99,1500.000,0.0,0.266000,http://f.wishabi.net/page_items/164122524/1565366422/extra_large.jpg
32,Chalo FreshCo,asparagus,2.99,NaN,0.0,0.659000,http://f.wishabi.net/page_items/163299378/1564749336/extra_large.jpg
25,Chalo FreshCo,bean,1.19,540.000,0.0,0.220370,http://f.wishabi.net/page_items/163299402/1564749341/extra_large.jpg
0,Chalo FreshCo,bread,2.27,675.000,0.0,0.336296,http://f.wishabi.net/page_items/163299388/1564749344/extra_large.jpg
28,Chalo FreshCo,bread,1.99,500.000,0.0,0.398000,http://f.wishabi.net/page_items/163299385/1564749342/extra_large.jpg
20,Chalo FreshCo,carrot,1.99,1360.776,0.0,0.146240,http://f.wishabi.net/page_items/164122511/1565366416/extra_large.jpg
19,Chalo FreshCo,cheese,5.99,900.000,0.0,0.665556,http://f.wishabi.net/page_items/163299438/1564749338/extra_large.jpg
36,Chalo FreshCo,corn,2.99,220.000,0.0,1.359091,http://f.wishabi.net/page_items/163299401/1564749341/extra_large.jpg


In [224]:
df_agg


price_100                                              
                                                                                  clean_image_url current_price price_100    weight2 weight_num
merchant_name  product                                                                                                                         
Chalo FreshCo  apple         http://f.wishabi.net/page_items/163299386/1564749343/extra_large.jpg          1.29  0.284000        NaN        0.0
               apricot       http://f.wishabi.net/page_items/164122515/1565366418/extra_large.jpg          3.99  0.166333   1500.000        0.0
               asparagus     http://f.wishabi.net/page_items/163299378/1564749336/extra_large.jpg          2.99  0.659000        NaN        0.0
               bean          http://f.wishabi.net/page_items/163299402/1564749341/extra_large.jpg          1.19  0.220370    540.000        0.0
               bread         http://f.wishabi.net/page_items/163299385/1564749342/extra_large.jpg          1.99  0.336296    500.000        0.0
               carrot        http://f.wishabi.net/page_items/164122511/1565366416/extra_large.jpg          1.99  0.146240   1360.776        0.0
               cheese        http://f.wishabi.net/page_items/163299438/1564749338/extra_large.jpg          5.99  0.665556    900.000        0.0
               corn          http://f.wishabi.net/page_items/163299401/1564749341/extra_large.jpg          2.99  1.359091    220.000        0.0
               cucumber      http://f.wishabi.net/page_items/163299379/1564749337/extra_large.jpg          0.69       NaN        NaN        1.0
               fish          http://f.wishabi.net/page_items/163299447/1564749345/extra_large.jpg          6.99  1.541000        NaN        0.0
               grape         http://f.wishabi.net/page_items/163299377/1564749335/extra_large.jpg          1.88       NaN        NaN        0.0
               green pepper  http://f.wishabi.net/page_items/164122513/1565366417/extra_large.jpg          1.49  0.002205  67585.208        0.0
               lemon         http://f.wishabi.net/page_items/163299380/1564749338/extra_large.jpg          0.40       NaN        NaN        1.0
               lettuce       http://f.wishabi.net/page_items/164122519/1565366420/extra_large.jpg          0.99       NaN        NaN        1.0
               meat          http://f.wishabi.net/page_items/163299361/1564749336/extra_large.jpg          3.99  0.880000    600.000        0.0
               mushroom      http://f.wishabi.net/page_items/163299376/1564749334/extra_large.jpg          1.69  0.744493    227.000        0.0
               onion         http://f.wishabi.net/page_items/164122511/1565366416/extra_large.jpg          0.69  0.146240   1360.776        0.0
               orange        http://f.wishabi.net/page_items/163299384/1564749341/extra_large.jpg          3.49  0.256471   1360.776        0.0
               pasta         http://f.wishabi.net/page_items/163299434/1564749335/extra_large.jpg          0.99  0.145588    680.000        0.0
               pear          http://f.wishabi.net/page_items/163299389/1564749345/extra_large.jpg          1.49  0.328000        NaN        0.0
               plum          http://f.wishabi.net/page_items/164122524/1565366422/extra_large.jpg          3.99  0.266000   1500.000        0.0
               rice          http://f.wishabi.net/page_items/163299397/1564749338/extra_large.jpg          1.29  0.781818    165.000        0.0
               tomato        http://f.wishabi.net/page_items/163299375/1564749333/extra_large.jpg          0.99  0.218000    540.000        0.0
               watermelon    http://f.wishabi.net/page_items/164122506/1565366414/extra_large.jpg          3.99  0.079968   4989.512        0.0
               yogurt        http://f.wishabi.net/page_items/163299433/1564749334/extra_large.jpg          4.99  0.665333    750.000        0.0
Food Basics    apple         http://f.wishabi.net/page_i

In [211]:
# df_sort_food2.loc[(df_sort_food2.clean_image_url=='http://f.wishabi.net/page_items/163299379/1564749337/extra_large.jpg')]
df_agg2=df_agg.groupby(['merchant_name']).agg({'current_price':sum})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


KeyError: ignored

In [209]:
df_agg2['count']=df_agg.groupby(['merchant_name']).count()

NameError: ignored

In [210]:
df_agg2['Overal_percentage']=df_agg.groupby(['merchant_name']).count()*100/len(df_sort_food3["product"].unique())

NameError: ignored

In [0]:
df_agg2

,current_price,count,Overal_percentage
merchant_name,,,
Chalo FreshCo,105.37,28,63.636364
Food Basics,96.95,27,61.363636
Foodland Ontario,197.48,35,79.545455
FreshCo,128.81,35,79.545455
Giant Tiger,42.26,7,15.909091
Healthy Planet,23.76,3,6.818182
Loblaws,285.01,56,127.272727
Longos,512.95,63,143.181818
M&M Food Market,243.54,19,43.181818
